**Прогноз цен на авиабилеты**

Датасет:
https://www.kaggle.com/competitions/aviachipta-narxini-bashorat-qilish/data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_data = pd.read_csv("train_data.csv", index_col='id')
test_data = pd.read_csv("test_data.csv", index_col='id')
submission = pd.read_csv("sample_solution.csv")
df = train_data

df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [5]:
le = LabelEncoder()
train_data['airline'] = le.fit_transform(train_data['airline'])
train_data['source_city'] = le.fit_transform(train_data['source_city'])
train_data['destination_city'] = le.fit_transform(train_data['destination_city'])
train_data['class'] = le.fit_transform(train_data['class'])
train_data['stops'] = le.fit_transform(train_data['stops'])
train_data['class'] = le.fit_transform(train_data['class'])
train_data['departure_time'] = le.fit_transform(train_data['departure_time'])
train_data['arrival_time'] = le.fit_transform(train_data['arrival_time'])

In [6]:
train_data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,5,UK-810,0,1,0,5,5,1,14.25,21,7212
2,4,SG-5094,3,2,2,5,4,1,1.75,7,5292
3,5,UK-846,0,4,0,2,2,0,9.58,5,60553
4,5,UK-706,4,4,0,2,3,1,6.75,28,5760
5,3,6E-5394,1,1,2,4,5,1,2.00,4,10712


In [7]:
test_data['airline'] = le.fit_transform(test_data['airline'])
test_data['source_city'] = le.fit_transform(test_data['source_city'])
test_data['destination_city'] = le.fit_transform(test_data['destination_city'])
test_data['class'] = le.fit_transform(test_data['class'])
test_data['stops'] = le.fit_transform(test_data['stops'])
test_data['class'] = le.fit_transform(test_data['class'])
test_data['departure_time'] = le.fit_transform(test_data['departure_time'])
test_data['arrival_time'] = le.fit_transform(test_data['arrival_time'])

In [8]:
test_data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,1,AI-765,4,2,0,5,2,0,28.25,2
2,5,UK-747,2,1,0,5,5,0,13.83,34
3,1,AI-570,5,1,2,1,1,0,2.00,30
4,0,I5-974,3,5,0,3,2,1,5.17,26
5,1,AI-770,4,5,0,0,5,1,16.33,35


In [9]:
X_train = train_data.drop('price', axis=1)
y_train = train_data['price']

In [10]:
train_set, test_set = train_test_split(train_data, test_size=0.2, random_state=42)

In [11]:
num_attribs=['airline', 'stops', 'class', 'duration', 'days_left', 'source_city', 'departure_time', 'arrival_time', 'destination_city']
cat_attribs=['flight']

num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

In [12]:
X_prep = full_pipeline.fit_transform(X_train)
X_prep.toarray()[0:5,:]

array([[ 1.03113236, -0.4306786 ,  0.67396452, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48472127,  2.55289329,  0.67396452, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.03113236, -0.4306786 , -1.48375764, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.03113236, -0.4306786 ,  0.67396452, ...,  0.        ,
         0.        ,  0.        ],
       [-0.06168981,  2.55289329,  0.67396452, ...,  0.        ,
         0.        ,  0.        ]])

In [13]:
test_prep = full_pipeline.transform(test_data)

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

LR_model = LinearRegression()
LR_model.fit(X_prep, y_train)
lr_predict=LR_model.predict(X_prep)
mse=mean_squared_error(lr_predict,y_train)
error=np.sqrt(mse)
print(f"MSE: {error.round()}")

y_test_predicted = LR_model.predict(test_prep)
y_test_predicted

MSE: 6271.0


array([54646.51402719, 58035.27010939, 40245.3766178 , ...,
       51997.73343156, 47694.85595815, 51371.49629243])

In [15]:
DTR_model = DecisionTreeRegressor()
DTR_model.fit(X_prep, y_train)
lr_predict=DTR_model.predict(X_prep)
mse=mean_squared_error(lr_predict,y_train)
error=np.sqrt(mse)
print(f"MSE: {error.round()}")

y_test_predicted2 = DTR_model.predict(test_prep)
y_test_predicted2

MSE: 42.0


array([54128., 59033., 23484., ..., 50271., 50868., 66063.])

In [16]:
RF_model = RandomForestRegressor()
RF_model.fit(X_prep, y_train)
lr_predict=RF_model.predict(X_prep)
mse=mean_squared_error(lr_predict,y_train)
error=np.sqrt(mse)
print(f"MSE: {error.round()}")

y_test_predicted3 = RF_model.predict(test_prep)
y_test_predicted3

MSE: 1241.0


array([53945.54, 65065.31, 23262.66, ..., 50271.  , 50400.59, 60043.54])

In [17]:
submission['price'] = y_test_predicted2
submission

,id,price
0,1,54128.0
1,2,59033.0
2,3,23484.0
3,4,2228.0
4,5,4969.0
...,...,...
4995,4996,60978.0
4996,4997,4499.0
4997,4998,50271.0
4998,4999,50868.0


In [18]:
submission.to_csv('submission.csv', index=False)

In [19]:
import pickle

filename = 'decision_tree_model.pkl'
pickle.dump(DTR_model, open(filename, 'wb'))

In [20]:
import joblib

joblib.dump(full_pipeline, "full_pipeline.pkl")

['full_pipeline.pkl']

**ТЕСТ**

Пример того, как модель принимает на вход переменные

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import pickle
import joblib

In [22]:
model = 'decision_tree_model.pkl'
loaded_model = pickle.load(open(model, 'rb'))

In [30]:
full_pipeline = joblib.load("full_pipeline.pkl")

In [24]:
id_fligth = 1
airline = "Air_India"
flight = "AI-765"
source_city = "Kolkata"
departure_time = "Evening"
stops = "one"
arrival_time = "Night"
destination_city = "Delhi"
class_flight = "Business"
duration = 28.25
days_left = 2

df1 = pd.DataFrame([[airline, flight, source_city, departure_time, stops, arrival_time, destination_city, class_flight, duration, days_left]],
                   columns=["airline", "flight", "source_city", "departure_time", "stops", "arrival_time", "destination_city", "class", "duration", "days_left"])
df1.index.name = 'id'

In [25]:
import pandas as pd

df1 = pd.DataFrame(df1)

df1.to_csv('file.csv')

In [26]:
test1 = pd.read_csv('file.csv', index_col='id')

#test1 = pd.read_json('file.json', orient='records')

In [27]:
le = LabelEncoder()
test1['airline'] = le.fit_transform(test1['airline'])
test1['source_city'] = le.fit_transform(test1['source_city'])
test1['destination_city'] = le.fit_transform(test1['destination_city'])
test1['class'] = le.fit_transform(test1['class'])
test1['stops'] = le.fit_transform(test1['stops'])
test1['class'] = le.fit_transform(test1['class'])
test1['departure_time'] = le.fit_transform(test1['departure_time'])
test1['arrival_time'] = le.fit_transform(test1['arrival_time'])

In [31]:
X_test_prep1 = full_pipeline.transform(test1)
y_predicted = loaded_model.predict(X_test_prep1)

y_predicted

array([50385.])